# Larger Scale PCA Example and the Limits of PCA

The previous notebook discussed the mechanics of PCA by looking at a two-dimensional dataset.  PCA is generally more valuable when applied to high-dimensional data.  This notebook will illustrate the use of PCA to reduce the dimensionity of image data.


## PCA on MNIST Digits

In [ ]:
#%matplotlib notebook

import os
import tensorflow_datasets as tfds
import numpy as np
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
import sklearn.datasets



def load_mnist(split='train'):
   
    dataset = tfds.load(name="mnist", split=split)
    ds_numpy = tfds.as_numpy(dataset)
        
    images = np.array([ex['image'] for ex in ds_numpy])
    dataset = tfds.load(name="mnist", split=split)
    ds_numpy = tfds.as_numpy(dataset)
    labels = np.array([ex['label'] for ex in ds_numpy])

    # Rescale and convert to floats...
    result_images = np.array(images / 255.0, dtype=np.float32)
    result_labels = np.array(labels, dtype=np.float32)

    return result_images, result_labels



### Load the MNIST dataset:

In [ ]:
X, labels = load_mnist()
print(X.shape)
X = X.reshape(X.shape[0], -1)
print(X.shape)

## Scree Plots

Execute the cell below to visualize the amount of variance captured by each of the principal components for the MNIST data set.  Around 90% of the variance is captured by the first 100 principal components. 

In [ ]:
print(X.shape)
pca = PCA()
pca.fit(X)

plt.plot(pca.explained_variance_ratio_, '.-')
plt.title('PC Variance')
plt.figure()
plt.plot(np.cumsum(pca.explained_variance_ratio_), '.-')
plt.title('Cumulative PC Variance')

## Visualizing the First Two Principal Components

The figure below shows all of the 0's, 1's and 2's in the data set projected onto the first two principal components. 

In [ ]:
Z = pca.transform(X)
#plt.plot(X_low[:,0], X_low[:,1], '.')
plt.plot(Z[labels==0, 0], Z[labels==0, 1],'.r', alpha=.1)
plt.plot(Z[labels==1, 0], Z[labels==1, 1],'.g', alpha=.1)
plt.plot(Z[labels==2, 0], Z[labels==2, 1],'.b', alpha=.1)
plt.show()

## PCA For Image Compression

In [ ]:
pca = PCA(n_components=50)
pca.fit(X)
Z50 = pca.transform(X)
X_back = pca.inverse_transform(Z50)

In [ ]:
X_back = X_back.reshape(-1, 28, 28)
X_img = X.reshape(-1, 28, 28)
plt.imshow(X_img[0,:,:],cmap='gray')
plt.figure()
plt.imshow(X_back[0,:,:],cmap='gray')



## Where Does PCA Fall Short?

Consider the following dataset.  Take a second to predict what will happen when we use PCA to project this to one dimension.

In [ ]:
X1 = np.random.multivariate_normal([-5, 3], [[1.5, 3], [3, 6.5]], 200)
X2 = np.random.multivariate_normal([-6.5, 5], [[1.5, 3], [3, 6.5]], 200)
X = np.vstack((X1, X2))
np.random.shuffle(X)
plt.plot(X[:, 0], X[:, 1], '.')
plt.axis('equal')
plt.show()

In [ ]:
pca = PCA(n_components=1)
pca.fit(X)
Z1 = pca.transform(X)
X_back = pca.inverse_transform(Z1)
np.random.shuffle(X)
plt.plot(X_back[:, 0], X_back[:, 1], '.')
plt.axis('equal')
plt.show()

Moral of the story: The directions of maximum variance aren't always the most meaningful dimensions.

## Nonlinear Structure

Execute the cell below to visualize some additional data with structure that is not well captured by PCA.

In [ ]:
moons = sklearn.datasets.make_moons(n_samples=1000, noise=.05)[0]
circles = sklearn.datasets.make_circles(n_samples=1000, factor=.5,
                                noise=.05)[0]

plt.plot(moons[:, 0], moons[:, 1], '.')
plt.figure()

plt.plot(circles[:, 0], circles[:, 1], '.')

from mpl_toolkits.mplot3d import Axes3D

fig = plt.figure()
ax = Axes3D(fig)
X, color = sklearn.datasets.make_swiss_roll(n_samples=1500)
ax.scatter(X[:, 0], X[:, 1], X[:, 2], c=color, cmap=plt.cm.Spectral)

## PCA is a linear method

Linear methods *can* rotate, shear, scale, project, but they *can't* unroll.

For that we need non-linear dimensionality reduction techniques:

https://github.com/jakevdp/PythonDataScienceHandbook/blob/master/notebooks/05.10-Manifold-Learning.ipynb
